In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from torchvision.models import resnet50
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
from torch.utils.data import ConcatDataset

# TASK 1

In [107]:
# Define transformations for the training dataset
train_transforms = transforms.Compose([
    # Convert image from PIL format to tensor
    transforms.PILToTensor(),
    # Change image type to float for compatibility with PyTorch
    transforms.ConvertImageDtype(torch.float),
    # Resize images to 224x224 pixels
    transforms.Resize((224, 224)),
    # Normalize images using mean and std for each color channel
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define transformations for the test dataset (same as training)
test_transforms = transforms.Compose([
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Resize((224, 224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the training dataset with specified transformations
train_dataset = datasets.Flowers102(root='data', split='train', transform=train_transforms, download=True)

# Load the validation dataset with test transformations
val_dataset = datasets.Flowers102(root='data', split='val', transform=test_transforms, download=True)

# Load the test dataset with test transformations
test_dataset = datasets.Flowers102(root='data', split='test', transform=test_transforms, download=True)

In [108]:
# Create a DataLoader for the training dataset
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)

# Create a DataLoader for the validation dataset (no shuffling)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)

# Create a DataLoader for the test dataset (no shuffling)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

In [109]:
# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [96]:
import matplotlib.pyplot as plt

# Get one batch of data from train_loader
inputs, classes = next(iter(train_loader))

# Set the figure size for the plot
plt.figure(figsize=(30, 16), dpi=1000)

# Use the actual batch size
batch_size = inputs.size(0)

for i in range(batch_size):  # Use the batch size instead of a fixed value
    plt.subplot(4, 8, i + 1)  # Adjust the number of rows and columns based on the batch size
    plt.imshow(inputs[i].numpy().transpose((1, 2, 0)))  # Convert tensor to image format
    plt.title(f'Class: {classes[i].item()}')  # Add title with the class label
    plt.axis('off')  # Turn off the axes

# Display the images
plt.show()

In [110]:
class FlowerCNN(nn.Module):
    def __init__(self):
        super(FlowerCNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # Output: 32 x 224 x 224
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)                   # Reduces size by half
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1) # Output: 64 x 112 x 112
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1) # Output: 128 x 56 x 56

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 512)                           # Input size after pooling
        self.fc2 = nn.Linear(512, 102)                                     # Output for 102 classes

    def forward(self, x):
        # Forward pass through the network
        x = self.pool(F.relu(self.conv1(x)))   # After conv1 and pool
        x = self.pool(F.relu(self.conv2(x)))   # After conv2 and pool
        x = self.pool(F.relu(self.conv3(x)))   # After conv3 and pool

        x = x.view(-1, 128 * 28 * 28)          # Flatten for fully connected layers
        x = F.relu(self.fc1(x))                 # First fully connected layer
        x = self.fc2(x)                         # Output layer

        return x

In [111]:
# Model initialization and sending to the device
model_cnn = FlowerCNN().to(device)

In [112]:
# Initialize model and optimizer

# Hyperparameters
epochs = 10  # Number of training epochs
lr = 0.001   # Learning rate

# Optimizer
optimizer = optim.SGD(model_cnn.parameters(), lr=lr)  # Stochastic Gradient Descent

# Loss function
criterion = nn.CrossEntropyLoss()  # For multi-class classification

# TensorBoard for logging
writer = SummaryWriter(log_dir="runs/flowers_experiment")  # Log directory for experiments

# Training function
def train(model, train_loader, criterion, optimizer):
    model.train()  # Set model to training mode
    running_train_loss = 0.0  # Accumulate training loss
    train_correct = 0  # Count correct predictions
    train_total = 0  # Total examples processed
    train_preds, train_labels = [], []  # Store predictions and true labels

    for inputs, labels in train_loader:  # Iterate over batches
        optimizer.zero_grad()  # Reset gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass to compute gradients
        optimizer.step()  # Update model parameters

        running_train_loss += loss.item()  # Accumulate loss
        _, predicted = torch.max(outputs, 1)  # Get predicted classes

        train_total += labels.size(0)  # Update total examples
        train_correct += (predicted == labels).sum().item()  # Count correct predictions
        train_preds.extend(predicted.cpu().numpy())  # Store predictions
        train_labels.extend(labels.cpu().numpy())    # Store true labels

    # Calculate average loss and metrics after training epoch
    train_loss = running_train_loss / len(train_loader)  # Average training loss
    train_accuracy = 100 * train_correct / train_total   # Training accuracy in percentage
    train_f1 = f1_score(train_labels, train_preds, average="weighted")  # F1 score

    return train_loss, train_accuracy, train_f1  # Return training results

# Evaluation function for validation set
def evaluate(model, val_loader, criterion):
    model.eval()  # Set model to evaluation mode
    running_val_loss = 0.0  # Accumulate validation loss
    val_correct = 0  # Count correct predictions
    val_total = 0   # Total examples processed
    val_preds, val_labels = [], []  # Store predictions and true labels

    with torch.no_grad():  # Disable gradient calculation for efficiency
        for inputs, labels in val_loader:  # Iterate over validation batches
            outputs = model(inputs)   # Forward pass
            loss = criterion(outputs, labels)   # Compute loss

            running_val_loss += loss.item()   # Accumulate loss
            _, predicted = torch.max(outputs, 1)   # Get predicted classes

            val_total += labels.size(0)   # Update total examples
            val_correct += (predicted == labels).sum().item()   # Count correct predictions
            val_preds.extend(predicted.cpu().numpy())   # Store predictions
            val_labels.extend(labels.cpu().numpy())     # Store true labels

    # Calculate average loss and metrics after evaluation
    val_loss = running_val_loss / len(val_loader)   # Average validation loss
    val_accuracy = 100 * val_correct / val_total     # Validation accuracy in percentage
    val_f1 = f1_score(val_labels, val_preds, average="weighted")   # F1 score

    return val_loss, val_accuracy, val_f1   # Return evaluation results

In [113]:
# Loop over the specified number of epochs
for epoch in range(1, epochs + 1):
    print(f"\nEpoch {epoch}/{epochs}")  # Print current epoch number

    # Training phase
    train_loss, train_accuracy, train_f1 = train(model_cnn, train_loader, criterion, optimizer)

    # Evaluation phase
    val_loss, val_accuracy, val_f1 = evaluate(model_cnn, val_loader, criterion)

    # Log metrics to TensorBoard
    writer.add_scalar('Loss/train', train_loss, epoch)  # Log training loss
    writer.add_scalar('Accuracy/train', train_accuracy, epoch)  # Log training accuracy
    writer.add_scalar('F1/train', train_f1, epoch)  # Log training F1 score
    writer.add_scalar('Loss/val', val_loss, epoch)  # Log validation loss
    writer.add_scalar('Accuracy/val', val_accuracy, epoch)  # Log validation accuracy
    writer.add_scalar('F1/val', val_f1, epoch)  # Log validation F1 score

    # Print metrics for both training and validation sets
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Train F1: {train_f1:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%, Validation F1: {val_f1:.4f}")

writer.close()  # Close the TensorBoard writer


Epoch 1/10
Train Loss: 4.6274, Train Accuracy: 0.69%, Train F1: 0.0011
Validation Loss: 4.6195, Validation Accuracy: 1.96%, Validation F1: 0.0054

Epoch 2/10
Train Loss: 4.6188, Train Accuracy: 1.86%, Train F1: 0.0061
Validation Loss: 4.6145, Validation Accuracy: 2.45%, Validation F1: 0.0102

Epoch 3/10
Train Loss: 4.6103, Train Accuracy: 2.25%, Train F1: 0.0068
Validation Loss: 4.6089, Validation Accuracy: 2.16%, Validation F1: 0.0106

Epoch 4/10
Train Loss: 4.6014, Train Accuracy: 3.92%, Train F1: 0.0207
Validation Loss: 4.6027, Validation Accuracy: 3.43%, Validation F1: 0.0227

Epoch 5/10
Train Loss: 4.5915, Train Accuracy: 3.82%, Train F1: 0.0313
Validation Loss: 4.5954, Validation Accuracy: 5.10%, Validation F1: 0.0261

Epoch 6/10
Train Loss: 4.5794, Train Accuracy: 7.35%, Train F1: 0.0477
Validation Loss: 4.5864, Validation Accuracy: 6.08%, Validation F1: 0.0313

Epoch 7/10
Train Loss: 4.5653, Train Accuracy: 9.51%, Train F1: 0.0632
Validation Loss: 4.5756, Validation Accuracy: 

**Improved model with Adam Optimizer**

In [114]:
class ImprovedFlowerCNN(nn.Module):
    def __init__(self):
        super(ImprovedFlowerCNN, self).__init__()

        # Convolutional layers with Batch Normalization and Dropout
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.dropout3 = nn.Dropout(0.5)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 102)

    def forward(self, x):
        x = self.bn1(self.pool(F.relu(self.conv1(x))))
        # x = self.dropout1(x)

        x = self.bn2(self.pool(F.relu(self.conv2(x))))
        # x = self.dropout2(x)

        x = self.bn3(self.pool(F.relu(self.conv3(x))))


        x = x.view(-1, 128 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        # x = self.dropout_fc(x)  # Apply dropout before final layer
        x = self.fc2(x)

        return x

In [115]:
# Device configuration: Automatically set to CUDA if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model initialization and sending to the device
model_cnn_improved = ImprovedFlowerCNN().to(device)

# TensorBoard for logging
writer = SummaryWriter(log_dir="runs/flowers_improved")

optimizer = optim.Adam(model_cnn_improved.parameters(), lr=lr)

In [116]:
# Training loop for improved baseline model
for epoch in range(1, epochs + 1):
    print(f"\nEpoch {epoch}/{epochs}")

    # Training phase for improved CNN
    train_loss_improved, train_accuracy_improved, train_f1_improved = train(model_cnn_improved, train_loader, criterion, optimizer)
    # Evaluation phase for improved CNN
    val_loss_improved, val_accuracy_improved, val_f1_improved = evaluate(model_cnn_improved, val_loader, criterion)

    # Log metrics to TensorBoard (same as before)
    writer.add_scalar('Loss/train/improved', train_loss_improved, epoch)
    writer.add_scalar('Accuracy/train/improved', train_accuracy_improved, epoch)
    writer.add_scalar('F1/train/improved', train_f1_improved, epoch)
    writer.add_scalar('Loss/val/improved', val_loss_improved, epoch)
    writer.add_scalar('Accuracy/val/improved', val_accuracy_improved, epoch)
    writer.add_scalar('F1/val/improved', val_f1_improved, epoch)

    print(f"Train Loss Improved: {train_loss_improved:.4f}, Train Accuracy Improved: {train_accuracy_improved:.2f}%, Train F1 Improved: {train_f1_improved:.4f}")
    print(f"Validation Loss Improved: {val_loss_improved:.4f}, Validation Accuracy Improved: {val_accuracy_improved:.2f}%, Validation F1 Improved: {val_f1_improved:.4f}")

writer.close()


Epoch 1/10
Train Loss Improved: 11.9526, Train Accuracy Improved: 1.96%, Train F1 Improved: 0.0180
Validation Loss Improved: 6.7556, Validation Accuracy Improved: 2.65%, Validation F1 Improved: 0.0160

Epoch 2/10
Train Loss Improved: 5.8531, Train Accuracy Improved: 5.29%, Train F1 Improved: 0.0480
Validation Loss Improved: 6.0553, Validation Accuracy Improved: 2.94%, Validation F1 Improved: 0.0151

Epoch 3/10
Train Loss Improved: 5.2631, Train Accuracy Improved: 8.43%, Train F1 Improved: 0.0836
Validation Loss Improved: 6.2669, Validation Accuracy Improved: 3.43%, Validation F1 Improved: 0.0248

Epoch 4/10
Train Loss Improved: 4.9942, Train Accuracy Improved: 9.61%, Train F1 Improved: 0.1062
Validation Loss Improved: 6.8990, Validation Accuracy Improved: 3.24%, Validation F1 Improved: 0.0199

Epoch 5/10
Train Loss Improved: 4.7540, Train Accuracy Improved: 11.47%, Train F1 Improved: 0.1238
Validation Loss Improved: 5.3895, Validation Accuracy Improved: 2.84%, Validation F1 Improved: 

**Improved model without Adam Optimizer**

In [117]:
# Device configuration: Automatically set to CUDA if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model initialization and sending to the device
model_cnn_improved = ImprovedFlowerCNN().to(device)

# TensorBoard for logging
writer = SummaryWriter(log_dir="runs/flowers_improved2")

optimizer = optim.SGD(model_cnn_improved.parameters(), lr=lr)

In [118]:
# Training loop for improved baseline model
for epoch in range(1, epochs + 1):
    print(f"\nEpoch {epoch}/{epochs}")

    # Training phase for improved CNN
    train_loss_improved, train_accuracy_improved, train_f1_improved = train(model_cnn_improved, train_loader, criterion, optimizer)
    # Evaluation phase for improved CNN
    val_loss_improved, val_accuracy_improved, val_f1_improved = evaluate(model_cnn_improved, val_loader, criterion)

    # Log metrics to TensorBoard (same as before)
    writer.add_scalar('Loss/train/improved', train_loss_improved, epoch)
    writer.add_scalar('Accuracy/train/improved', train_accuracy_improved, epoch)
    writer.add_scalar('F1/train/improved', train_f1_improved, epoch)
    writer.add_scalar('Loss/val/improved', val_loss_improved, epoch)
    writer.add_scalar('Accuracy/val/improved', val_accuracy_improved, epoch)
    writer.add_scalar('F1/val/improved', val_f1_improved, epoch)

    print(f"Train Loss Improved: {train_loss_improved:.4f}, Train Accuracy Improved: {train_accuracy_improved:.2f}%, Train F1 Improved: {train_f1_improved:.4f}")
    print(f"Validation Loss Improved: {val_loss_improved:.4f}, Validation Accuracy Improved: {val_accuracy_improved:.2f}%, Validation F1 Improved: {val_f1_improved:.4f}")

writer.close()


Epoch 1/10
Train Loss Improved: 4.4814, Train Accuracy Improved: 2.84%, Train F1 Improved: 0.0241
Validation Loss Improved: 4.1348, Validation Accuracy Improved: 10.98%, Validation F1 Improved: 0.0701

Epoch 2/10
Train Loss Improved: 3.5046, Train Accuracy Improved: 29.61%, Train F1 Improved: 0.2913
Validation Loss Improved: 3.7622, Validation Accuracy Improved: 15.49%, Validation F1 Improved: 0.1116

Epoch 3/10
Train Loss Improved: 2.6762, Train Accuracy Improved: 53.43%, Train F1 Improved: 0.5249
Validation Loss Improved: 3.4954, Validation Accuracy Improved: 22.25%, Validation F1 Improved: 0.1870

Epoch 4/10
Train Loss Improved: 1.9629, Train Accuracy Improved: 73.53%, Train F1 Improved: 0.7320
Validation Loss Improved: 3.3429, Validation Accuracy Improved: 26.18%, Validation F1 Improved: 0.2247

Epoch 5/10
Train Loss Improved: 1.4177, Train Accuracy Improved: 84.41%, Train F1 Improved: 0.8438
Validation Loss Improved: 3.2238, Validation Accuracy Improved: 29.31%, Validation F1 Imp

In [119]:
from torchvision import models

In [120]:
# Load pre-trained ResNet50
resnet50 = models.resnet50(pretrained=True)

# Replace the last layer for 102 classes
resnet50.fc = nn.Linear(resnet50.fc.in_features, 102)

# Move model to device (GPU/CPU)
resnet50 = resnet50.to(device)

# Enable gradients for the last layer
for param in resnet50.fc.parameters():
    param.requires_grad = True

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [121]:
writer = SummaryWriter(log_dir="runs/flowers_improved_resnet")

In [122]:
optimizer = optim.SGD(resnet50.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    print(f"\nEpoch {epoch}/{epochs}")

    train_loss_resnet, train_accuracy_resnet, train_f1_resnet = train(resnet50, train_loader, criterion, optimizer)

    val_loss_resnet, val_accuracy_resnet, val_f1_resnet = evaluate(resnet50, val_loader, criterion)

    writer.add_scalar('Loss/train ResNet50', train_loss_resnet, epoch)
    writer.add_scalar('Accuracy/train ResNet50', train_accuracy_resnet, epoch)
    writer.add_scalar('F1/train ResNet50', train_f1_resnet, epoch)
    writer.add_scalar('Loss/val ResNet50', val_loss_resnet, epoch)
    writer.add_scalar('Accuracy/val ResNet50', val_accuracy_resnet, epoch)
    writer.add_scalar('F1/val ResNet50', val_f1_resnet, epoch)

    print(f"Train Loss: {train_loss_resnet:.4f}, Train Accuracy: {train_accuracy_resnet:.2f}%, Train F1: {train_f1_resnet:.4f}")
    print(f"Validation Loss: {val_loss_resnet:.4f}, Validation Accuracy: {val_accuracy_resnet:.2f}%, Validation F1: {val_f1_resnet:.4f}")

writer.close()


Epoch 1/10
Train Loss: 4.6279, Train Accuracy: 0.98%, Train F1: 0.0037
Validation Loss: 4.5726, Validation Accuracy: 1.18%, Validation F1: 0.0084

Epoch 2/10
Train Loss: 4.5303, Train Accuracy: 2.75%, Train F1: 0.0152
Validation Loss: 4.4939, Validation Accuracy: 4.02%, Validation F1: 0.0297

Epoch 3/10
Train Loss: 4.4409, Train Accuracy: 7.06%, Train F1: 0.0560
Validation Loss: 4.4192, Validation Accuracy: 8.53%, Validation F1: 0.0707

Epoch 4/10
Train Loss: 4.3496, Train Accuracy: 14.51%, Train F1: 0.1213
Validation Loss: 4.3429, Validation Accuracy: 14.41%, Validation F1: 0.1151

Epoch 5/10
Train Loss: 4.2633, Train Accuracy: 24.71%, Train F1: 0.2128
Validation Loss: 4.2663, Validation Accuracy: 20.88%, Validation F1: 0.1718

Epoch 6/10
Train Loss: 4.1617, Train Accuracy: 36.37%, Train F1: 0.3191
Validation Loss: 4.1832, Validation Accuracy: 28.04%, Validation F1: 0.2380

Epoch 7/10
Train Loss: 4.0705, Train Accuracy: 43.92%, Train F1: 0.3940
Validation Loss: 4.0958, Validation Acc

# Report on Flower Classification Using Convolutional Neural Networks and Transfer Learning

**Goal**

The primary goal of this project is to implement a flower classification Convolutional Neural Network (CNN) trained on the Flowers102 dataset. This classification system aims to ensure that flower images displayed on online floristry and delivery platforms accurately match their respective descriptions or categories. By achieving high accuracy in flower classification, we can enhance customer satisfaction and retention in the floristry market.

**Approach for Training the Model**

**Data Loading & Preprocessing**

Dataset: The Flowers102 dataset was loaded using torchvision.datasets, which contains 102 categories of flowers.
Transformations: The images were preprocessed using a series of transformations:
Conversion from PIL format to tensor.
Resizing images to 224x224 pixels.
Normalization using pre-defined mean and standard deviation values for each color channel.

**Baseline Model Implementation**

Model Architecture: A baseline CNN architecture was implemented with three convolutional layers followed by max pooling, and two fully connected layers. The architecture included ReLU as the activation function.

**Baseline Model Training**

Training Process: The baseline model was trained for 10 epochs using Stochastic Gradient Descent (SGD) with a learning rate of 0.001.
Logging: Training metrics, including loss, accuracy, and F1 score, were logged to TensorBoard for visualization and analysis.

**Model Evaluation**

Evaluation Metrics: The model's performance was evaluated based on training and validation loss, accuracy, and F1 score after each epoch.

**Baseline Model Improvement**

Techniques Used: To improve the model's performance, several techniques were applied:
Batch Normalization: Added after each convolutional layer to stabilize learning and improve convergence.
Dropout: Introduced dropout layers to reduce overfitting by randomly setting a fraction of input units to zero during training.

**Transfer Learning**

Pre-trained Model: The ResNet50 architecture was utilized for transfer learning. The last layer was replaced to adapt the model for classifying the 102 flower categories.
Training: The ResNet50 model was fine-tuned on the Flowers102 dataset, leveraging its pre-trained weights for improved performance.

**Attained ML Model Results**

**Baseline Model Performance**

The results from the baseline CNN model indicated ineffective training, particularly in the early epochs. Over 10 epochs, training accuracy only reached 10.88%, with an F1 score of 0.0698. Validation accuracy remained low at around 7.25%, suggesting that the baseline model struggled to generalize effectively.

**Improved Model Performance (Using Adam)**

The improved model utilizing the Adam optimizer showed slight enhancements over the baseline model but still yielded low training accuracy (maximum of 17.55%) and validation accuracy (up to 4.22%). This indicates that while Adam can improve convergence speed, the architecture still lacked sufficient capacity for effective classification.

**Improved Model Performance (Without Adam)**

The improved model without using Adam exhibited significant gains compared to previous attempts. Training accuracy surged to 99.12% by the end of 10 epochs, with an F1 score reaching 0.9911. Validation accuracy also increased notably to 31.67%, indicating successful generalization on validation data.

**ResNet50 Performance**

Utilizing the pre-trained ResNet50 model yielded the best results among all tested models. Training accuracy reached 62.35%, while validation accuracy climbed to 49.71%. The F1 score also improved significantly to 0.4511, confirming the effectiveness of transfer learning as ResNet50 could extract more complex features from flower images due to its deep architecture and prior training on a large dataset.


**Conclusion**

Architecture Effectiveness: The simple CNN failed to achieve satisfactory results, highlighting the importance of selecting an appropriate architecture for image classification tasks.
Optimization Impact: Different optimizers (SGD vs. Adam) demonstrated variations in convergence speed and overall performance.
Transfer Learning Advantage: The ResNet50 model showcased significantly better results due to leveraging pre-trained weights and a more complex architecture, making it a preferred choice for image classification tasks.
Overall, the project successfully demonstrated the implementation of a CNN for flower classification while showcasing significant improvements through both baseline enhancements and transfer learning techniques. These findings underscore the importance of careful model tuning and method selection to achieve high performance in image classification challenges.
